In [1]:
### Don't mind about this 
import warnings
warnings.filterwarnings('ignore')
###

%matplotlib inline 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier # I will use the Sklearn to get kNN
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
import math
from statistics import mode

In [2]:
q2_dataset = pd.read_csv('Q2.csv') #put the data in pd Frame
q2_dataset.head()

Color     Type  Doors      Tires Class
0    Red      SUV      2  Whitewall     +
1   Blue  Minivan      4  Whitewall     -
2  Green      Car      4  Whitewall     -
3    Red  Minivan      4  Blackwall     -
4  Green      Car      2  Blackwall     +

In [3]:
q2_dataset.shape

(14, 5)

In [4]:
q2_dataset['Class'].unique()

array(['+', '-'], dtype=object)

In [5]:
# Change + to 1 and - to 0
q2_dataset['Class'] = q2_dataset['Class']=='+'

In [6]:
q2_dataset['Class'] = q2_dataset['Class'].astype(int)

In [7]:
q2_dataset.head()

Color     Type  Doors      Tires  Class
0    Red      SUV      2  Whitewall      1
1   Blue  Minivan      4  Whitewall      0
2  Green      Car      4  Whitewall      0
3    Red  Minivan      4  Blackwall      0
4  Green      Car      2  Blackwall      1

In [8]:
# Now split the Xs from the Label 

Y = q2_dataset['Class']
X = q2_dataset.iloc[:,:-1]

In [9]:
print(X.shape)
print(Y.shape)

(14, 4)
(14,)


In [10]:
Y.unique()

array([1, 0])

## Preprocessing the dataset

### we can create one hot encoding on all features, as they are just 4 features. 


In [11]:
X = pd.get_dummies(X, columns=['Color', 'Type', 'Doors', 'Tires'])
X

Color_Blue  Color_Green  Color_Red  Type_Car  Type_Minivan  Type_SUV  \
0            0            0          1         0             0         1   
1            1            0          0         0             1         0   
2            0            1          0         1             0         0   
3            0            0          1         0             1         0   
4            0            1          0         1             0         0   
5            0            1          0         0             0         1   
6            1            0          0         0             0         1   
7            1            0          0         1             0         0   
8            0            0          1         0             0         1   
9            1            0          0         1             0         0   
10           0            1          0         0             0         1   
11           0            0          1         1             0         0   
12           0            1          0         0             0         1   
13           0            1          0         0             1         0   

    Doors_2  Doors_4  Tires_Blackwall  Tires_Whitewall  
0         1        0                0                1  
1         0        1                0                1  
2         0        1                0                1  
3         0        1                1                0  
4         1        0                1                0  
5         0        1                1                0  
6         1        0                1                0  
7         1        0                0                1  
8         1        0                1                0  
9         0        1                1                0  
10        0        1                0                1  
11        1        0                1                0  
12        1        0                1                0  
13        0        1                0                1

## Splitting the data to train and validation 
### we will not need to do any randomness, as the question mentioned that the last 3 examples are for validation

In [16]:
X_train = X.iloc[:-3,]
Y_train = Y.iloc[:-3,]
X_val = X.iloc[-3:,]
Y_val = Y.iloc[-3:,]

In [17]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)


(11, 10)
(11,)
(3, 10)
(3,)


## Convert the arrays to numpy arrays

In [18]:
X_train = X_train.values.reshape(11,10)
Y_train = Y_train.values.reshape(11,1)
X_val = X_val.values.reshape(3,10)
Y_val = Y_val.values.reshape(3,1)


## Implemening the Logistic Regrssion
### I completed on what I have done in the lab

In [19]:
type(Y_train)

numpy.ndarray

In [105]:
class LogisticReg:
    ## TODO
    # write a constructor that takes the learning rate and the number of iterations
    # save these params as a class members
    def __init__(self, learning_rate=0.01, iterations=100):
        self.theta= None
        ## TODO
        self.learning_rate=learning_rate
        self.iterations=iterations
      
    
    def _sigmoid(self,x):
        return 1/(1+np.exp(-x))   
        
    
    def _loss(self,X, y):
        
        # giving X and y calculate the loss function and return the value
        
        epsilon = 1e-5
        
        m = X.shape[0]# get the number of samples
        
        self.yhat = self._sigmoid((X @ self.theta))# calculate yhat
        

        J = (-1/m)*np.sum(y*np.log(self.yhat+epsilon) + ( 1 - y)*np.log(1-self.yhat+epsilon))
        
        return J
    
    
    def fit(self,X,y):
        
        
        #add column of ones for the bias
        
        #To add the bias 
        x_instance,x_feature = X.shape

        X = np.hstack((np.ones((x_instance,1)),X))

        # Make the Weights random at first
        
        self.theta = np.random.rand(x_feature+1,1)
        
                
        self.cost = np.zeros((self.iterations,1)) # initialize the cost
        self.cost[0] = self._loss(X,y)
        # perform GD
        for i in range(self.iterations): # loop over the number of iterations
                        
            # update theta
            self.theta =self.theta + self.learning_rate*(X.T@(y-self.yhat))
            
            self.cost[i] =  self._loss(X,y)# calculate the cost

        return self,self.cost
    

    
    def predict(self,x_test):
        m = x_test.shape[0]
        x_test = np.hstack((np.ones((m,1)),x_test))
        out = []
        for x_t in x_test:
            out.append(np.round(self._sigmoid(x_t @ self.theta)))
        
        return np.array(out)
                    
    def evaluate(self,x_test,y_test):
        
        predictions = self.predict(x_test)
        predictions[predictions >= 0.5] =1
        predictions[predictions < 0.5] =0
        count=0
        
        for i in range(len(predictions)):
            if predictions[i] == y_test[i]:
                count+=1
             
        return count/len(predictions)
                
        
        

In [106]:
cle = LogisticReg()

loss = cle.fit(X_train,Y_train)
y_predicted = cle.predict(X_val)
print(y_predicted)

print("The accuracy of this model is: " ,cle.evaluate(X_val,Y_val)*100,"%")

[[0.]
 [0.]
 [0.]]
The accuracy of this model is:  66.66666666666666 %


In [107]:
loss

(<__main__.LogisticReg at 0x7f24d3a63b50>,
 array([[1.42452014],
        [1.3382606 ],
        [1.25730483],
        [1.18185171],
        [1.11200819],
        [1.04778392],
        [0.98909287],
        [0.93576169],
        [0.88754343],
        [0.84413434],
        [0.80519187],
        [0.77035188],
        [0.73924368],
        [0.71150227],
        [0.68677747],
        [0.66474019],
        [0.64508627],
        [0.62753825],
        [0.6118457 ],
        [0.59778453],
        [0.58515554],
        [0.57378262],
        [0.56351075],
        [0.55420388],
        [0.54574298],
        [0.53802404],
        [0.53095638],
        [0.52446095],
        [0.51846895],
        [0.51292047],
        [0.50776337],
        [0.50295228],
        [0.49844768],
        [0.49421518],
        [0.49022476],
        [0.48645029],
        [0.48286894],
        [0.47946076],
        [0.47620833],
        [0.47309637],
        [0.47011153],
        [0.46724205],
        [0.46447764],
        [0.

In [108]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(Y_val, y_predicted))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3

